In [1]:
import dotenv
import os
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openai

In [11]:
# Configuração da API do OpenAI
dotenv.load_dotenv(dotenv.find_dotenv())
API_KEY = os.getenv("API_KEY")
openai.api_key = API_KEY

In [4]:
# Configuração do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\gustavo.ferreira\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [13]:
# Função para listar todos os arquivos PDF em um diretório
def listar_pdfs_em_diretorio(diretorio):
    return [os.path.join(diretorio, arquivo) for arquivo in os.listdir(diretorio) if arquivo.lower().endswith('.pdf')]

In [15]:
# Função para converter múltiplos PDFs em texto puro
def extrair_textos_dos_pdfs(diretorio):
    lista_caminhos_pdf = listar_pdfs_em_diretorio(diretorio)
    textos = []
    for caminho_pdf in lista_caminhos_pdf:
        texto = ""
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            for pagina in leitor_pdf.pages:
                texto += pagina.extract_text()
        textos.append(texto)
    return textos

In [6]:
# Função para pré-processamento de texto: tokenizar, remover stop words e stemizar o texto
def preprocessar_texto(texto):
    # Tokenização
    tokens = word_tokenize(texto.lower(), language='portuguese')
    # Remoção de stopwords e tokens vazios
    stopwords_pt = set(stopwords.words('portuguese'))
    tokens_filtrados = [token for token in tokens if token.isalnum() and token not in stopwords_pt]
    # Stemming - reduzir palavras focando em seu significado
    stemmer = RSLPStemmer()
    tokens_stemizados = [stemmer.stem(token) for token in tokens_filtrados]
    return ' '.join(tokens_stemizados)

In [7]:
def construir_indice(documentos):
    tfidf_vectorizer = TfidfVectorizer()
    matriz_tfidf = tfidf_vectorizer.fit_transform(documentos)
    return tfidf_vectorizer, matriz_tfidf

In [8]:
# Função para busca de documentos relevantes utilizando EMBEDDINGS (VETORIZAÇÃO DAS PALAVRAS)
# Aproxima o vetor gerado da pergunta com os vetores do documento referência para retornar a resposta 
def buscar_documentos(pergunta, documentos, tfidf_vectorizer, matriz_tfidf):
    pergunta = preprocessar_texto(pergunta)
    pergunta_vetor = tfidf_vectorizer.transform([pergunta]) 
    similaridades = cosine_similarity(pergunta_vetor, matriz_tfidf)
    indice_doc_relevantes = similaridades.argmax(axis=1)[0]
    return documentos[indice_doc_relevantes]

In [9]:
# Geração de respostas com o ChatGPT
def gerar_resposta(pergunta, contexto):
    resposta = openai.Completion.create(
        engine="davinci-002",
        prompt=f"Pergunta: {pergunta}\nContexto: {contexto}\nResposta:"
    )
    return resposta.choices[0].text.strip()

In [18]:
# 1. Pré-processamento de Dados
diretorio_pdfs = r'C:\Users\gustavo.ferreira\Desktop\IA_chatBot\Políticas'  # Substitua pelo caminho do diretório onde os PDFs estão localizados
textos = extrair_textos_dos_pdfs(diretorio_pdfs)

for texto in textos:
    # Verifica se o texto extraído não está vazio
    if texto.strip() == "":
        print("O texto extraído de um dos PDFs está vazio. Verifique o PDF.")
        continue

    texto_preprocessado = preprocessar_texto(texto)
    # print(texto_preprocessado)

polít alç alg empreend particip polít dat vers 02 conselh administr gustav matsumot validad indetermin document públic alg empreend particip págin 1 5 obje apresent regr alç aprov alg empreend particip fil pontu respons polít aplic csc alg atu nom benefíci diret alg polít dev consider conjunt códig condut alg estatut soc contrat s ocial empr document norm corporativ legisl aplic conjunt norte alç adot grup alg princípi códig condut alg polít regul princípi integr previst códig condut alg refer códig condut alg acord acion estatut soc alg polít corpor trans part relacion conflit inter polít corporativ g consequ polít corpor finanç estratég polít corpor juríd norm corpor alien ativ general polít dev analis conjunt estatut soc alg avali compet órg governanç assemble conselh administr diret event alç prev nest document limit atu dest polít dev cumpr conjunt regr prev códig condut alg estatut soc contrat soc empr polít norm instruç trabalh substitu aprov porvent poss exig outr norm específi

In [12]:
# Função principal
def principal():
    # 1. Pré-processamento de Dados
    diretorio_pdfs = 'C:\Users\gustavo.ferreira\Desktop\IA_chatBot\Políticas'  # Substitua pelo caminho do diretório onde os PDFs estão localizados
    textos = extrair_textos_dos_pdfs(diretorio_pdfs)

    for texto in textos:
        # Verifica se o texto extraído não está vazio
        if texto.strip() == "":
            print("O texto extraído de um dos PDFs está vazio. Verifique o PDF.")
            continue

        texto_preprocessado = preprocessar_texto(texto)
    # 2. Indexação de Conteúdo
    documentos = [texto_preprocessado]
    tfidf_vectorizer, matriz_tfidf = construir_indice(documentos)

    while True:
        # 3. Compreensão de Perguntas
        pergunta = input("Faça uma pergunta sobre os normativos corporativos (ou digite 'sair' para encerrar): ")
        if pergunta.lower() == 'sair':
            print("Até logo!")
            break

        # 4. Busca e Recuperação de Informações
        # Aproxima o vetor gerado da pergunta com os vetores do documento referência para retornar a resposta 
        doc_relevante = buscar_documentos(pergunta, documentos, tfidf_vectorizer, matriz_tfidf)

        # 5. Geração de Respostas
        resposta = gerar_resposta(pergunta, doc_relevante)
        print("Resposta:", resposta)

if __name__ == "__main__":
    principal()

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.